In [179]:
import pandas as pd
import numpy as np
import datetime
# from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 16)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_close_clean = 'df_close_clean'

In [180]:
d =  {'col':np.arange(0, 10, 1, dtype=int)}
df = pd.DataFrame(data=d)
df


,col
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [181]:
print(f'len(df): {len(df)}')
# df.iloc[10].empty
# df.iloc[len(df)-1:len(df)]
# df.iloc[0:len(df)]

len(df): 10


In [182]:

def _3_random_slices(
    len_df, n_samples, days_lookback, days_eval, verbose=False
):
    """Returns a list of random tuples of start_train, end_train, end_eval, where
    iloc[start_train:end_train] is used for training,
    and iloc[end_train:end_eval] is used for evaluation.  The length of the
    list is equal to n_samples.
    i.e. [(248, 368, 388), (199, 319, 339), ... (45, 165, 185)]

    Args:
        len_df(int): length of dataframe
        n_samples(int): number of slices to return
        days_lookback(int):  number of days to lookback for training
        days_eval(int): number of days forward for evaluation

    Return:
        r_slices(list of tuples): list of random tuples of start_train,
        end_train, end_eval, where iloc[start_train:end_train] is used for
        training, and iloc[end_train:end_eval] is used for evaluation.
          i.e. [(248, 368, 388), (199, 319, 339), ... (45, 165, 185)]
    """
    # v1 correct out-of-bound end_eval
    # v2 2023-03-11 check df generated from the random slices 

    # import random
    from random import randint

    # random.seed(0)
    n_sample = 0
    days_total = days_lookback + days_eval
    if verbose:
        print(
            f"days_lookback: {days_lookback}, days_eval: {days_eval}, days_total: {days_total}, len_df: {len_df}"
        )

    if days_total > len_df:    
        msg_err = f"days_total: {days_total} must be less or equal to len_df: {len_df}"
        raise SystemExit(msg_err)

    # random slices of iloc for train and eval that fits the constraints:
    #  days_lookback + days_eval > len_df
    #  start_train >= 0
    #  end_train - start_train = days_lookback
    #  end_eval - end_train = days_eval
    #  end_eval <= len_df
    r_slices = []
    while n_sample < n_samples:
        n_rand = randint(0, len_df)
        start_train = n_rand - days_lookback
        end_train = n_rand
        # start_eval = n_rand
        end_eval = n_rand + days_eval
        if 0 <= start_train and end_eval <= len_df:              
            r_slices.append((start_train, end_train, end_eval))
            n_sample += 1

    return r_slices



In [198]:
len_df = len(df)
verbose = True  # True prints more output
n_samples = 20
# days_lookbacks = [1, 2, 3]
days_lookbacks = [5]
days_lookbacks.sort()
days_eval = 0
print(f"len_df: {len_df}")
print(f'n_samples: {n_samples}')
print(f'days_lookbacks: {days_lookbacks}')
print(f'days_eval: {days_eval}')
print(f'verbose : {verbose }')

len_df: 10
n_samples: 20
days_lookbacks: [5]
days_eval: 0
verbose : True


In [199]:
max_lookback_slices = _3_random_slices(len_df, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval, verbose=True)

days_lookback: 5, days_eval: 0, days_total: 5, len_df: 10


In [200]:
max_lookback_slices 

[(3, 8, 8),
 (5, 10, 10),
 (2, 7, 7),
 (4, 9, 9),
 (2, 7, 7),
 (1, 6, 6),
 (0, 5, 5),
 (4, 9, 9),
 (2, 7, 7),
 (3, 8, 8),
 (4, 9, 9),
 (1, 6, 6),
 (2, 7, 7),
 (1, 6, 6),
 (5, 10, 10),
 (0, 5, 5),
 (4, 9, 9),
 (0, 5, 5),
 (3, 8, 8),
 (0, 5, 5)]

In [202]:
print(f'df[0:5]: {df[0:5]}\n')
# print(f'df[6:10]: {df[6:10]}')
# print(f'df[10:10]: {df[10:10]}')
print(f'df[9:9]: {df[9:9]}\n')
print(f'df[7:10]: {df[7:10]}\n')

df[0:5]:    col
0    0
1    1
2    2
3    3
4    4

df[9:9]: Empty DataFrame
Columns: [col]
Index: []

df[7:10]:    col
7    7
8    8
9    9

